In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import plot_model

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Plot the model
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)





You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [28]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError

# Read the CSV files
df_val = pd.read_csv(
    "/home/arush/GW_Project_1/Data_Generation/Continous_Check/validation_data.csv"
)
df_test = pd.read_csv(
    "/home/arush/GW_Project_1/Data_Generation/Continous_Check/test_data.csv"
)

# Image size
img_size = (469, 369)

# ImageDataGenerator for scaling
scale = ImageDataGenerator(rescale=1.0 / 255)

# Validation data generator
val_generator = scale.flow_from_dataframe(
    dataframe=df_val,
    x_col="Path",
    y_col="Time",
    target_size=img_size,
    class_mode="raw",  # For regression, use 'raw'
    batch_size=32,
    shuffle=True,
)

# Test data generator
test_generator = scale.flow_from_dataframe(
    dataframe=df_test,
    x_col="Path",
    y_col="Time",
    target_size=img_size,
    class_mode="raw",  # For regression, use 'raw'
    batch_size=32,
    shuffle=False,
)

Found 3394 validated image filenames.
Found 3395 validated image filenames.


In [42]:
from keras.src.legacy.saving import legacy_h5_format
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError

filepath = "/home/arush/GW_Project_1/Model_Training/Models/cnn_model_time_v1.h5"

# Load the model with custom objects
# Load the model
model = legacy_h5_format.load_model_from_hdf5(filepath, custom_objects={"mse": "mse"})


# Recompile the model with a fresh optimizer and metrics
model.compile(
    optimizer="adam",
    loss=MeanSquaredError(),
    metrics=[MeanSquaredError(), MeanAbsoluteError()],
)


results = model.evaluate(test_generator)

# Print evaluation results
print(f"Test Loss (MSE): {results[0]}")
print(f"Test Mean Squared Error: {results[1]}")
print(f"Test Mean Absolute Error: {results[2]}")

107/107 ━━━━━━━━━━━━━━━━━━━━ 101s 936ms/step - loss: 0.7675 - mean_absolute_error: 0.7645 - mean_squared_error: 0.7675
Test Loss (MSE): 0.7545260787010193
Test Mean Squared Error: 0.7558409571647644
Test Mean Absolute Error: 0.7545260787010193


In [43]:
# Print accuracy
print(results)
######

[0.7545260787010193, 0.7558409571647644, 0.7545260787010193]
